## Interactive version:

#### Requirements:
To correctly run the script <code>qiskit</code>, <code>matplotlib</code>, <code>pylatexenc</code>, <code>ipynb</code> and <code>ipywidgets</code> have to be installed. 

#### Inputs:
<ol>
    <li> <code>numbers</code>: list of integers </li>
    <li> <code>solution</code>: the number we want our sums to be equal to </li>
    <li> <code>method</code>: a string selecting the method to create combinations </li>
        </ol>

## Import libraries:
(tools.ipynb has to be in the same folder as adder)

In [1]:
from qiskit import QuantumCircuit, Aer, execute, QuantumRegister

import numpy as np
from qiskit.visualization import plot_histogram

from ipynb.fs.full.tools import adder, qft_1, count, w, sel
from ipywidgets import interact_manual, widgets

## The entire algorithm:

In [2]:
style = {'displaycolor': {'qtf-1':   ('#34495E', '#FFFFFF'),
                          (' SEL '):   ('#E67E22', '#FFFFFF'),
                          'h':   ('#2980B9', '#FFFFFF'),},
         'displaytext' : {'qtf-1': "\quad \mathrm{QFT}^{\dagger}."},
         'gatefacecolor' : '#27AE60',
         'gatetextcolor' : '#FFFFFF'}

def find_that(numbers, solution, method):
    #convert 'numbers' to a list of int
    numbers = list(map(int, numbers.split()))
    # filter numbers bigger than the final solution we want
    numbers = list(filter(lambda x: x <= solution, numbers))

    # compute the number of required qubits
    sum_num = sum(numbers)
    bit_sum = int(np.log2(sum_num)) + 1
    n = len(numbers) + bit_sum

    qr = QuantumRegister(len(numbers), 'q')
    qa = QuantumRegister(bit_sum, 'a')
    qc = QuantumCircuit(qr, qa)

    # implement the SEL gate
    qc.sel(len(numbers), method)
    # implement the Hadamard transform
    qc.h(qa)
    # implement c-P gates
    qc.adder(bit_sum, numbers)
    # implement the QFT-1
    qc.qft_1(qa)
    # extract the measure
    counts = count(qc)
    # filter states equal to 'solution'
    filtered = [(k[bit_sum:], v) for k, v in counts.items() if k[:bit_sum] == ('{0:0'+str(bit_sum)+'b}').format(solution)]

    str_final = 'Solutions: '
    for sol in filtered:
        combination = []
        for i in range(len(numbers)):
            if sol[0][i] == '1':
                combination.append(str(numbers[::-1][i]))
        str_final = str_final + '\n> ' + '+'.join(combination)

    qc.draw(output='mpl', style=style)
    
    return print(str_final)

## Insert the set of `numbers` and the `method` we want to use to find `solution` :

In [3]:
%matplotlib

Using matplotlib backend: <object object at 0x7f85b0847ff0>


In [4]:
i = interact_manual(find_that, solution=widgets.IntSlider(min=1, max=50, step=1, value=16), 
                    numbers='5 7 8 9', 
                    method=['hilbert', 'w state', 'w state-x'])

interactive(children=(Text(value='5 7 8 9', description='numbers'), IntSlider(value=16, description='solution'…

In [5]:
import qiskit.tools.jupyter
%qiskit_version_table